# Homework 2:  Improve Baseline CNN model and compute metrics for assessing the performance of the CNN-based model

## Notebook Outline

    1. Train Basic Model (From Homework 1)

    2. Saving and Loading Model
    
    3. Metrics Access Performance
    
    4. Hyper-parameter Tuning
    
    5. Overfitting Prevention
    
    6. Compare Performance of Basic and Improved Model

## 1. Train Basic Model (From Homework 1)

In [6]:
# Load the tensorflow, which is a framework for deep learning.
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
# Load numpy library as "np", which can handle large matrices and provides some mathematical functions.
import numpy as np 
# Load pandas as "pd", which is useful when working with data tables. 
import pandas as pd 
# Load random, which provide some randomize functions.
import random
# Load a function pyplot as "plt" to plot figures.
import matplotlib.pyplot as plt
# Load functions to calculate precision, and recall
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Setup the random seed for reproducibility
seed = 1234
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [7]:
# The root directory of the pmuBAGE data
pmuBAGE_data_dir = "../data/pmuBAGE/data"

# Number of the tensor for voltage and frequency
voltage_tensor_number = 31
frequency_tensor_number = 21

# Load each tensors of voltage events and concatenate them as a big tensor.
voltage_tensor_list = []
for idx in range(voltage_tensor_number):
    voltage_sub_tensor = np.load(f"{pmuBAGE_data_dir}/voltage/voltage_{idx}.npy")
    voltage_tensor_list.append(voltage_sub_tensor)
voltage_tensor = np.concatenate(voltage_tensor_list, axis=0)


# Load each tensors of frequency events and concatenate them as a big tensor.
frequency_tensor_list = []
for idx in range(frequency_tensor_number):
    frequency_sub_tensor = np.load(f"{pmuBAGE_data_dir}/frequency/frequency_{idx}.npy")
    frequency_tensor_list.append(frequency_sub_tensor)
frequency_tensor = np.concatenate(frequency_tensor_list, axis=0)

# Transpose the big tensor as (event_idx, timestamp, PMU_idx, measurements)
voltage_tensor = np.transpose(voltage_tensor, (0, 3, 2, 1))
frequency_tensor = np.transpose(frequency_tensor, (0, 3, 2, 1))

# Print the shape of the voltage event
print(voltage_tensor.shape)
print(frequency_tensor.shape)

(620, 600, 100, 4)
(84, 600, 100, 4)


In [8]:
"""
    Use standardization to pre-process the pmu time series data.
    Input  -> two original tensors: voltage_tensor, frequency_tensor
    Output -> two standardized tensors: voltage_tensor_standardized, frequency_tensor_standardized
    Requirement Details: 
        The tensor shape is (number_of_event, timestamps (600), pmus (100), measurements (4))
        For each time sequence (Single pmu measurement sequence, 600 timestamps), standardize them by Z-Score
        z-score = (x - mean) / std
"""

# Voltage tensor

voltage_mean = np.mean(voltage_tensor, axis=1)
voltage_mean = np.expand_dims(voltage_mean, axis=1)
voltage_std = np.std(voltage_tensor, axis=1)
voltage_std = np.expand_dims(voltage_std, axis=1)
voltage_tensor_standardized = np.nan_to_num((voltage_tensor - voltage_mean) / voltage_std)

# Frequency tensor

frequency_mean = np.mean(frequency_tensor, axis=1)
frequency_mean = np.expand_dims(frequency_mean, axis=1)
frequency_std = np.std(frequency_tensor, axis=1)
frequency_std = np.expand_dims(frequency_std, axis=1)
frequency_tensor_standardized = np.nan_to_num((frequency_tensor - frequency_mean) / frequency_std)

print(voltage_tensor_standardized.shape)
print(frequency_tensor_standardized.shape)


(620, 600, 100, 4)
(84, 600, 100, 4)


In [9]:
# Number of the classes
num_classes = 2

# Number of the voltage and frequency events in the dataset
n_voltage = voltage_tensor_standardized.shape[0]
n_frequency = frequency_tensor_standardized.shape[0]

# Define the labels
# Voltage events' label is defined as: 0
voltage_label = np.array([0] * n_voltage)
# Frequency events' label is defined as: 1
frequency_label = np.array([1] * n_frequency)

"""
    Implement the one-hot encoding on the lablel of of the voltage and frequency event labels.
    Input  -> Original voltage and frequency labels (voltage_label, frequency_label)
    Output -> One-hot encoded voltage and frequency labels (voltage_label_onehot, frequency_label_onthot)
    Voltage label: "0" -> "[1, 0]"
    Frequency label: "1" -> "[0, 1]"
    You can use any library or tool for doing this
"""

voltage_label_onehot = tf.keras.utils.to_categorical(voltage_label, num_classes=num_classes)
frequency_label_onthot = tf.keras.utils.to_categorical(frequency_label, num_classes=num_classes)

# Should be [1, 0]
print(voltage_label_onehot[0])
# Should be [0, 1]
print(frequency_label_onthot[0])
# Should be (620, 2)
print(voltage_label_onehot.shape)
# Should be (84, 2)
print(frequency_label_onthot.shape)

[1. 0.]
[0. 1.]
(620, 2)
(84, 2)


In [10]:
voltage_tensor_standarded_permuted = voltage_tensor_standardized[np.random.permutation(n_voltage)]
frequency_tensor_standarded_permuted = frequency_tensor_standardized[np.random.permutation(n_frequency)]

In [11]:
# Seperate the data to train and test
train_portion = 0.7

# Samples
X_voltage = voltage_tensor_standarded_permuted
X_frequency = frequency_tensor_standarded_permuted
# Labels
y_voltage = voltage_label_onehot
y_frequency = frequency_label_onthot

"""
    Seperate the samples and labels to train and test datasets.
    70% of the voltage and frequency samples and labels are combined as training dataset
    30% remainings are combined as testing dataset
    Input  -> X_voltage, X_frequency, y_voltage, y_frequency
    Output -> X_train, y_train, X_test, y_test
        X_train contains 70% of the X_voltage and X_frequency
        y_train contains 70% of the y_voltage and y_frequency
        X_test contains 30% of the X_voltage and X_frequency
        y_test contains 30% of the y_voltage and y_frequency
"""

# X_train
X_train_voltage = X_voltage[:int(n_voltage * train_portion)] 
X_train_frequency = X_frequency[:int(n_frequency * train_portion)]
X_train = np.concatenate([X_train_voltage, X_train_frequency], axis=0)

# y_train
y_train_voltage = y_voltage[:int(n_voltage * train_portion)] 
y_train_frequency = y_frequency[:int(n_frequency * train_portion)]
y_train = np.concatenate([y_train_voltage, y_train_frequency], axis=0)

# X_test
X_test_voltage = X_voltage[int(n_voltage * train_portion):] 
X_test_frequency = X_frequency[int(n_frequency * train_portion):]
X_test = np.concatenate([X_test_voltage, X_test_frequency], axis=0)

# y_test
y_test_voltage = y_voltage[int(n_voltage * train_portion):] 
y_test_frequency = y_frequency[int(n_frequency * train_portion):]
y_test = np.concatenate([y_test_voltage, y_test_frequency], axis=0)

# Should be (492, 600, 100, 4)
print(X_train.shape)
# Should be (492, 2)
print(y_train.shape)
# Should be (212, 600, 100, 4)
print(X_test.shape)
# Should be (212, 2)
print(y_test.shape)

(492, 600, 100, 4)
(492, 2)
(212, 600, 100, 4)
(212, 2)


In [21]:
def build_model():
    """
        Add more laybers in the model, at least three convolusional layers.
        Then add the Flatten and Dense layers to make the output same with the number of classes.
    """

    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(600, 100, 4)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

model = build_model()

# Define the Loss function
loss_func = tf.keras.losses.CategoricalCrossentropy()

# Define the optimizer and learning rate
lr = 0.01
optimizer = tf.keras.optimizers.Adam(lr)

# Compile the neural network model
model.compile(optimizer=optimizer, loss=loss_func, metrics=['categorical_accuracy'])

# Train the neural network
history = model.fit(X_train, y_train, epochs=10, batch_size=16)

# Evaluate the neural network
loss, accuracy = model.evaluate(X_test, y_test)
print(f"The accuracy of the neural network on the test dataset is: {accuracy}.")

## 2. Saving and Loading Model

In [14]:
##-----------------------------------------------------------------------##
##---------------------Students start filling below----------------------##
##-----------------------------------------------------------------------##

"""
    Save trained model to file for future application or further fine-tuning train. 
"""

# Write the code to save the trained model to file.
model.save('midrar_model.h5')


"""
    Load the model from the file and compile it.
"""
lr = 0.01
optimizer = tf.keras.optimizers.Adam(lr)
loss_func = tf.keras.losses.CategoricalCrossentropy()
# Write the code to load the model from file and compile it.
model = models.load_model('midrar_model.h5')
model.compile(optimizer=optimizer, loss=loss_func, metrics=['categorical_accuracy'])
##-----------------------------------------------------------------------##
##------------------------------End filling------------------------------##
##-----------------------------------------------------------------------##

## 3. Metrics Assess Performance

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 598, 98, 32)       1184      
                                                                 
 max_pooling2d (MaxPooling2  (None, 299, 49, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 297, 47, 32)       9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 148, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 146, 21, 32)       9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 73, 10, 32)        0

In [16]:
# Get the test samples and labels, and get the model's prediction on test data.
X_test = X_test
y_test = y_test
y_pred = model.predict(X_test)
print(y_pred)
print('\n\n----\n\n')
print(y_test)

7/7 [==============================] - 1s 185ms/step
[[0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.20202245]
 [0.79797757 0.2020224

In [33]:
##-----------------------------------------------------------------------##
##---------------------Students start filling below----------------------##
##-----------------------------------------------------------------------##

"""
    Homework 1, only calculate the accuracy of the whole dataset.
    In this task, you required to calculate the accuracy, precision, recall, and F1-score.
"""
from sklearn.preprocessing import label_binarize

# Accuracy

# The argmax returns the highest probability in the provided y_predict variable in the previous block cell. I can't run the accuracy directly because I'd be comparing binary values to numerical values. Therefore,
# converting one of the variables to binary would solve the problem, I think. 
y_prediction = tf.argmax(y_pred, axis=1)
y_testing = tf.argmax(y_test, axis=1)
accuracy = accuracy_score(y_true=y_testing, y_pred=y_prediction)

# Precision

# This is kind of confusing. But here is what I tried, without the average attribute, I would get a zero precision. As per scikit learn documentation (link provided in the slides), the average is specified for multi
# classification problem. What I don't understand is why wouldn't I use the other attributes, such as 'binary'.


precision = precision_score(y_true=y_testing, y_pred=y_prediction, average='weighted')

# Recall
recall = recall_score(y_true=y_testing, y_pred=y_prediction, average='weighted')

# F1-Score
f1 = f1_score(y_true=y_testing, y_pred=y_prediction, average='weighted')

print(f"The accuracy is: {accuracy}.")
print(f"The precision is: {precision}.")
print(f"The recall is: {recall}.")
print(f"The f1 score is: {f1}.")

##-----------------------------------------------------------------------##
##------------------------------End filling------------------------------##
##-----------------------------------------------------------------------##

The accuracy is: 0.8773584905660378.
The precision is: 0.7697579209683162.
The recall is: 0.8773584905660378.
The f1 score is: 0.8200436142979046.


/home/midrar/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 4. Hyper-parameter Tuning

In [22]:
"""
    Seperate the Training Dataset to Training (80%) and Validation (20%).
    Perform the hyper-parameter tuning to find the best parameter combination.
    
"""

val_portaton = 0.2

n_voltage_train = int(n_voltage * train_portion)
n_frequency_train = int(n_frequency * train_portion)

X_train_voltage = X_voltage[:int(n_voltage * train_portion)] 
X_train_frequency = X_frequency[:int(n_frequency * train_portion)]
X_train = np.concatenate([X_train_voltage, X_train_frequency], axis=0)

# X_val_hp
X_val_hp_voltage = X_train[:int(n_voltage_train * val_portaton)]
X_val_hp_frequency = X_train[n_voltage_train: n_voltage_train + int(n_frequency_train * val_portaton)]
X_val_hp = np.concatenate([X_val_hp_voltage, X_val_hp_frequency], axis=0)

# y_val_hp
y_val_hp_voltage = y_train[:int(n_voltage_train * val_portaton)]
y_val_hp_frequency = y_train[n_voltage_train: n_voltage_train + int(n_frequency_train * val_portaton)]
y_val_hp = np.concatenate([y_val_hp_voltage, y_val_hp_frequency], axis=0)


# X_train_hp
X_train_hp_voltage = X_train[int(n_voltage_train * val_portaton): n_voltage_train]
X_train_hp_frequency = X_train[n_voltage_train + int(n_frequency_train * val_portaton):]
X_train_hp = np.concatenate([X_train_hp_voltage, X_train_hp_frequency], axis=0)

# y_train_hp
y_train_hp_voltage = y_train[int(n_voltage_train * val_portaton): n_voltage_train]
y_train_hp_frequency = y_train[n_voltage_train + int(n_frequency_train * val_portaton):]
y_train_hp = np.concatenate([y_train_hp_voltage, y_train_hp_frequency], axis=0)


# Should be (492, 600, 100, 4)
print(X_train_hp.shape)
# Should be (492, 2)
print(y_train_hp.shape)
# Should be (212, 600, 100, 4)
print(X_val_hp.shape)
# Should be (212, 2)
print(y_val_hp.shape)

(395, 600, 100, 4)
(395, 2)
(97, 600, 100, 4)
(97, 2)


In [23]:
# Setup the random seed for reproducibility
seed = 1234
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [24]:
# Hyper-parameters
learning_rates = [0.001, 0.01]
batch_sizes = [16, 32]
training_epochs = [10, 20]

##-----------------------------------------------------------------------##
##---------------------Students start filling below----------------------##
##-----------------------------------------------------------------------##


"""
    Try different combination of the hyper-parameters.
    Train on training dataset, test on validation dataset.
    Choose the best hyper-parameter combination to train the final improved model.
"""

# I tried different parameters are in the cell block. 

best_hyperparameter = {"learning_rate": 0, "batch_size": 0, "training_epoch": 0}
best_accuracy_val = 0.0


for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for training_epoch in training_epochs:
            print(f"Current hyper-parameters: learning_rate: {learning_rate}, batch_size: {batch_size}, training_epoch: {training_epoch}.")
            
            """
                Train the model under hyper-parameter setting, and evaluate over validation dataset.
            """
            
            """ Filling code below """
            
            # build model
            model_tuning = build_model()
            # Train the model with the above hyper-parameters
            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

            model_tuning.compile(optimizer=optimizer, loss=loss_func, metrics=['categorical_accuracy'])

            history = model_tuning.fit(X_train_hp, y_train_hp, epochs=training_epoch, batch_size=batch_size, validation_data=(X_val_hp, y_val_hp))

            """ End Filling """
    
            # Evaluate the hyper-parameter tuning neural network
            loss_val, accuracy_val = model_tuning.evaluate(X_val_hp, y_val_hp)
            print(f"Current validation accuracy is: {accuracy_val}.\n")
            
            if accuracy_val > best_accuracy_val:
                best_accuracy_val = accuracy_val
                best_hyperparameter["learning_rate"] = learning_rate
                best_hyperparameter["batch_size"] = batch_size
                best_hyperparameter["training_epoch"] = training_epoch
            tf.keras.backend.clear_session()
            
            
print(f"best validation accuracy is: {best_accuracy_val}")
print(f"best hyper-parameter setting is: {best_hyperparameter}")

##-----------------------------------------------------------------------##
##------------------------------End filling------------------------------##
##-----------------------------------------------------------------------##

Current hyper-parameters: learning_rate: 0.001, batch_size: 16, training_epoch: 10.
Epoch 1/10
25/25 [==============================] - 14s 544ms/step - loss: 0.4414 - categorical_accuracy: 0.8658 - val_loss: 0.1959 - val_categorical_accuracy: 0.8866
Epoch 2/10
25/25 [==============================] - 14s 542ms/step - loss: 0.1999 - categorical_accuracy: 0.9443 - val_loss: 0.1509 - val_categorical_accuracy: 0.9691
Epoch 3/10
25/25 [==============================] - 14s 576ms/step - loss: 0.1376 - categorical_accuracy: 0.9494 - val_loss: 0.1454 - val_categorical_accuracy: 0.9691
Epoch 4/10
25/25 [==============================] - 13s 539ms/step - loss: 0.1044 - categorical_accuracy: 0.9722 - val_loss: 0.1014 - val_categorical_accuracy: 0.9691
Epoch 5/10
25/25 [==============================] - 14s 563ms/step - loss: 0.0654 - categorical_accuracy: 0.9722 - val_loss: 0.1717 - val_categorical_accuracy: 0.9588
Epoch 6/10
25/25 [==============================] - 14s 566ms/step - loss: 0.0393

## In the next cell block, I am trying different hyperparameters to see if this network can be improved:

In [ ]:
# Hyper-parameters

learning_rates = [0.0001, 0.003]
batch_sizes = [10, 20]
training_epochs = [15, 25]

##-----------------------------------------------------------------------##
##---------------------Students start filling below----------------------##
##-----------------------------------------------------------------------##


"""
    Try different combination of the hyper-parameters.
    Train on training dataset, test on validation dataset.
    Choose the best hyper-parameter combination to train the final improved model.
"""

# I tried different parameters are in the cell block. 

best_hyperparameter = {"learning_rate": 0, "batch_size": 0, "training_epoch": 0}
best_accuracy_val = 0.0


for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for training_epoch in training_epochs:
            print(f"Current hyper-parameters: learning_rate: {learning_rate}, batch_size: {batch_size}, training_epoch: {training_epoch}.")
            
            """
                Train the model under hyper-parameter setting, and evaluate over validation dataset.
            """
            
            """ Filling code below """
            
            # build model
            model_tuning = build_model()
            # Train the model with the above hyper-parameters
            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

            model_tuning.compile(optimizer=optimizer, loss=loss_func, metrics=['categorical_accuracy'])

            history = model_tuning.fit(X_train_hp, y_train_hp, epochs=training_epoch, batch_size=batch_size, validation_data=(X_val_hp, y_val_hp))

            """ End Filling """
    
            # Evaluate the hyper-parameter tuning neural network
            loss_val, accuracy_val = model_tuning.evaluate(X_val_hp, y_val_hp)
            print(f"Current validation accuracy is: {accuracy_val}.\n")
            
            if accuracy_val > best_accuracy_val:
                best_accuracy_val = accuracy_val
                best_hyperparameter["learning_rate"] = learning_rate
                best_hyperparameter["batch_size"] = batch_size
                best_hyperparameter["training_epoch"] = training_epoch
            tf.keras.backend.clear_session()
            
            
print(f"best validation accuracy is: {best_accuracy_val}")
print(f"best hyper-parameter setting is: {best_hyperparameter}")

##-----------------------------------------------------------------------##
##------------------------------End filling------------------------------##
##-----------------------------------------------------------------------##

In [26]:
lr = 0.001
batch_size = 16
training_epoch = 20

improved_model = build_model()

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
loss_func = tf.keras.losses.CategoricalCrossentropy()

improved_model.compile(optimizer=optimizer, loss=loss_func, metrics=['categorical_accuracy'])

improved_model_history = improved_model.fit(X_train, y_train, epochs=training_epoch, batch_size=batch_size)

# Evaluate the neural network
loss, accuracy = improved_model.evaluate(X_test, y_test)
print(f"The accuracy of the neural network on the test dataset is: {accuracy}.")


Epoch 1/20
31/31 [==============================] - 17s 525ms/step - loss: 0.4777 - categorical_accuracy: 0.8882
Epoch 2/20
31/31 [==============================] - 16s 519ms/step - loss: 0.1516 - categorical_accuracy: 0.9451
Epoch 3/20
31/31 [==============================] - 16s 505ms/step - loss: 0.1077 - categorical_accuracy: 0.9715
Epoch 4/20
31/31 [==============================] - 16s 513ms/step - loss: 0.1141 - categorical_accuracy: 0.9634
Epoch 5/20
31/31 [==============================] - 15s 496ms/step - loss: 0.0768 - categorical_accuracy: 0.9776
Epoch 6/20
31/31 [==============================] - 15s 487ms/step - loss: 0.0677 - categorical_accuracy: 0.9736
Epoch 7/20
31/31 [==============================] - 15s 500ms/step - loss: 0.0534 - categorical_accuracy: 0.9858
Epoch 8/20
31/31 [==============================] - 15s 486ms/step - loss: 0.0528 - categorical_accuracy: 0.9797
Epoch 9/20
31/31 [==============================] - 15s 487ms/step - loss: 0.0248 - categorical_

## 5. Overfitting Prevention
    Early Stopping to Prevent the overfitting.

In [28]:
from tensorflow.keras.callbacks import EarlyStopping
# Set the best hyper-parameter from previous tuning. (You may change them based on previous results)

learning_rate = 0.001
# batch_size = 32
batch_size = 16
training_epoch = 20

##-----------------------------------------------------------------------##
##---------------------Students start filling below----------------------##
##-----------------------------------------------------------------------##


"""
    Use the early stopping to prevent the overfitting.
    Useful resource: https://keras.io/api/callbacks/early_stopping/
"""


# Build model
improved_model = build_model()
# Define the loss function
loss_func = tf.keras.losses.CategoricalCrossentropy()
# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate)
# Compile the model
improved_model.compile(optimizer=optimizer, loss=loss_func, metrics=['categorical_accuracy'])

""" Filling code below """

# Define the early stopping callback
# Monitor = default value val_loss.
# Patience = number of epochs with no improvements after which training will be stopped.
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

""" End Filling """

# Train the model with the tuned hyper-parameters and early-stopping.
history = improved_model.fit(X_train_hp, y_train_hp, validation_data=(X_val_hp, y_val_hp), 
                             epochs=training_epoch, batch_size=batch_size, callbacks=[early_stopping])

##-----------------------------------------------------------------------##
##------------------------------End filling------------------------------##
##-----------------------------------------------------------------------##

Epoch 1/20
25/25 [==============================] - 15s 583ms/step - loss: 0.4077 - categorical_accuracy: 0.8886 - val_loss: 0.1795 - val_categorical_accuracy: 0.9485
Epoch 2/20
25/25 [==============================] - 14s 558ms/step - loss: 0.1768 - categorical_accuracy: 0.9392 - val_loss: 0.1401 - val_categorical_accuracy: 0.9691
Epoch 3/20
25/25 [==============================] - 13s 536ms/step - loss: 0.1423 - categorical_accuracy: 0.9595 - val_loss: 0.1892 - val_categorical_accuracy: 0.9485
Epoch 4/20
25/25 [==============================] - 14s 544ms/step - loss: 0.0862 - categorical_accuracy: 0.9772 - val_loss: 0.1373 - val_categorical_accuracy: 0.9278
Epoch 5/20
25/25 [==============================] - 13s 533ms/step - loss: 0.0622 - categorical_accuracy: 0.9797 - val_loss: 0.2188 - val_categorical_accuracy: 0.9485
Epoch 6/20
25/25 [==============================] - 13s 506ms/step - loss: 0.0630 - categorical_accuracy: 0.9772 - val_loss: 0.1893 - val_categorical_accuracy: 0.948

## 6. Compare Performance of Basic and Improved Model

In [29]:
# Precision, recall, F1-score of the basic model

y_pred = model.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
precision = precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='macro')
recall = recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='macro')

print("Performance of the basic model.")
print(f"The accuracy is: {accuracy}.")
print(f"The precision is: {precision}.")
print(f"The recall is: {recall}.")
print(f"The f1 score is: {f1}.")

7/7 [==============================] - 1s 186ms/step
Performance of the basic model.
The accuracy is: 0.8773584905660378.
The precision is: 0.4386792452830189.
The recall is: 0.5.
The f1 score is: 0.46733668341708545.


/home/midrar/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
# Precision, recall, F1-score of the improved model

y_pred_improved = improved_model.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_improved, axis=1))
precision = precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred_improved, axis=1), average='macro')
recall = recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred_improved, axis=1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred_improved, axis=1), average='macro')

print("Performance of the improved model.")
print(f"The accuracy is: {accuracy}.")
print(f"The precision is: {precision}.")
print(f"The recall is: {recall}.")
print(f"The f1 score is: {f1}.")

7/7 [==============================] - 2s 203ms/step
Performance of the improved model.
The accuracy is: 0.9198113207547169.
The precision is: 0.8042153377348908.
The recall is: 0.8715880893300247.
The f1 score is: 0.8326445321569538.


## I see what you're talking about now. The preceision in the model, before it is improved, was about 87% and the F-score was 47%. However, the improved model has an accuracy of 91% and F1 score of 83%. 
## Thank you